# APIM ❤️ OpenAI

## OpenAI Mock Server(s)

This notebook provides guidance on the use of mock servers locally and in Azure. The AI-Gateway Mock server is designed to mimic the behavior and responses of the OpenAI API, thereby creating an efficient simulation environment suitable for testing and development purposes on the integration with APIM and other use cases.
The [app.py](app.py) can be customized to tailor the Mock server to specific use cases.

### Run locally
Open a terminal and type:
```
pip install -r requirements.txt
flask --app app.py --debug run
```
### Deploy to Azure Web Apps
Prerequisites
- [Python 3.8 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)

Execute the following steps to deploy to Azure.
The ```mock_webapps``` is a list of names that will be used to create the Azure Web App(s). All the Web Apps will reuse the same App Service Plan.

### 0️⃣ Initialize notebook variables

In [1]:
tool_prefix="st-pevo1-"
mock_resource_group = tool_prefix + "lab-ai-gateway-mock"
#mock_location = "swedencentral" # the location that will be used for the resource group, app service plan and web apps
mock_location = "westeurope" # the location that will be used for the resource group, app service plan and web apps
mock_webapps = [ {"name": tool_prefix + "openaimock1"}, {"name": tool_prefix + "openaimock2"} ] # ensure that the names are not being used within Azure
mock_webapp_runtime = "PYTHON:3.12"
mock_app_service_plan = tool_prefix + "openaimock-plan"
mock_app_service_plan_sku = "B1"


In [2]:
import requests
url = "https://www.google.com" 
try: 
    r = requests.get(url, timeout=1) 
    r.raise_for_status() 
except requests.exceptions.HTTPError as errh: 
    print("HTTP Error") 
    print(errh.args[0]) 
# Prints the response code 
print(r) 

<Response [200]>


### 1️⃣ Deploy or update Mock Server(s)
The following command is an [all-in-one command](https://learn.microsoft.com/en-us/cli/azure/webapp?view=azure-cli-latest#az-webapp-up) that creates all the Azure resources and deploys the source code. When finished it should open a browser window for each Mock server created.
You can repeat this step if you modify the ```app.py``` file to incorporate your specific mocking behavior. This will allow you to redeploy using the same resources that were previously created. 

In [3]:
for mock_webapp in mock_webapps:
    mock_webapp_name = mock_webapp.get("name")
    ! az webapp up --resource-group {mock_resource_group} --name {mock_webapp_name} --location {mock_location} \
        --plan {mock_app_service_plan} --runtime {mock_webapp_runtime} --sku {mock_app_service_plan_sku} --track-status true --launch-browser 

{
  "appserviceplan": "st-pevo1-openaimock-plan",
  "location": "westeurope",
  "name": "st-pevo1-openaimock1",
  "os": "Linux",
  "resourcegroup": "st-pevo1-lab-ai-gateway-mock",
  "runtime_version": "PYTHON|3.12",
  "runtime_version_detected": "-",
  "sku": "BASIC",
  "src_path": "c:\\Users\\u3184\\azdevops\\pevosdcgh\\AI-Gateway\\tools\\mock-server"
}


D:\a\_work\1\s\build_scripts\windows\artifacts\cli\Lib\site-packages\paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
D:\a\_work\1\s\build_scripts\windows\artifacts\cli\Lib\site-packages\paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.


{
  "appserviceplan": "st-pevo1-openaimock-plan",
  "location": "westeurope",
  "name": "st-pevo1-openaimock2",
  "os": "Linux",
  "resourcegroup": "st-pevo1-lab-ai-gateway-mock",
  "runtime_version": "PYTHON|3.12",
  "runtime_version_detected": "-",
  "sku": "BASIC",
  "src_path": "c:\\Users\\u3184\\azdevops\\pevosdcgh\\AI-Gateway\\tools\\mock-server"
}


D:\a\_work\1\s\build_scripts\windows\artifacts\cli\Lib\site-packages\paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
D:\a\_work\1\s\build_scripts\windows\artifacts\cli\Lib\site-packages\paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.


### 2️⃣ Test the Mock server(s)
The following code uses the Python HTTP client library to simulate OpenAI requests to the Mock server(s). To guide the mock server towards simulating behaviors such as returning a specific status code or incorporating a delay to influence the response time, you may modify the content structure as illustrated below.

In [5]:
import json
import requests

for mock_webapp in mock_webapps:
    mock_webapp_name = mock_webapp.get("name")
    url = "https://" + mock_webapp_name + ".azurewebsites.net/openai/deployments/gpt5/chat/completions?api-version=2024-02-01"
    url = "https://" + mock_webapp_name + ".azurewebsites.net/openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-02-01"
    print(url)
    messages={
        "messages": [
            {
                "role": "system", 
                "content": {
                    "simulation": {
                        "default": {"response_status_code": 200, "wait_time_ms": 0}
                    }
                }
            }
        ]
    }
    messages2={
        "messages": [
            {
                "role": "system", 
                "content": "You are a sarcastic unhelpful assistant."
            },
            {"role": "user", 
            "content": "Can you tell me the time, please?"
            }
        ]
    }
    response = requests.post(url, json = messages)
    print("status code: ", response.status_code)
    print("headers: ", response.headers)
    if (response.status_code == 200):
        data = json.loads(response.text)
        print("response: ", data.get("choices")[0].get("message").get("content"))
    else:
        print(response.text)

https://st-pevo1-openaimock1.azurewebsites.net/openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-02-01
status code:  200
headers:  {'Content-Length': '325', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Fri, 18 Oct 2024 09:42:00 GMT', 'Server': 'gunicorn', 'Set-Cookie': 'ARRAffinity=5548126bacb825528497974a6c799e6f1387107738785077a880ce94fd374f1d;Path=/;HttpOnly;Secure;Domain=st-pevo1-openaimock1.azurewebsites.net, ARRAffinitySameSite=5548126bacb825528497974a6c799e6f1387107738785077a880ce94fd374f1d;Path=/;HttpOnly;SameSite=None;Secure;Domain=st-pevo1-openaimock1.azurewebsites.net', 'x-ratelimit-remaining-tokens': '5000', 'x-ratelimit-remaining-requests': '50', 'x-ms-region': 'st-pevo1-openaimock1.azurewebsites.net'}
response:  Mock response from st-pevo1-openaimock1.azurewebsites.net
https://st-pevo1-openaimock2.azurewebsites.net/openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-02-01
status code:  200
headers:  {'Content-Length': '325', 'Content-

### (optional) Stream logs

Execute the result of the following script in a terminal to tail the streamed logs

In [ ]:
for mock_webapp in mock_webapps:
    mock_webapp_name = mock_webapp.get("name")
    ! echo az webapp log tail --name {mock_webapp_name} --resource-group {mock_resource_group}

### 🗑️ Clean up resources

When you're finished with the APIM ❤️ OpenAI Mock server(s), you can remove all your deployed resources from Azure. To avoid extra charges and keep your Azure subscription uncluttered. Removing the resource group is the fastest way to remove all Azure resources that you have created.

In [ ]:
run_cell = True
if run_cell:
    ! az group delete --name {mock_resource_group} -y